In [1]:
import os
import threading
import psycopg2
from flask import Flask, request, jsonify, make_response
import logging
from io import StringIO
from flask_httpauth import HTTPBasicAuth
from urllib.parse import urlparse

In [2]:
conn = psycopg2.connect(host="postgres", dbname="crawldb", user="ieps", password="spiderman", port=5432)
conn.autocommit = True

lock = threading.Lock()

In [3]:
basic_auth = HTTPBasicAuth()

@basic_auth.verify_password
def verify_password(username, password):
    if password == "&*qRyQ-7dMCX$S9&":
        return username
    else:
        return None

@basic_auth.error_handler
def basic_auth_error(status):
    return jsonify({"success": False, "message": "Wrong credentials!"}), status

In [4]:
from flask import Flask
from flask import jsonify
from flask_httpauth import HTTPBasicAuth

app = Flask(__name__)

In [5]:
@app.errorhandler(404)
def not_found_error(error):
    return jsonify({"success": False, "message": "Object not found!"}), 404

@app.errorhandler(500)
def internal_error(error):
    # rollback db if using transactions
    return jsonify({"success": False, "message": "Server error!"}), 500

In [6]:
@app.route('/visited_domains', methods=['GET'])
@basic_auth.login_required
def visited_domains():

    cur = conn.cursor()
    cur.execute("SELECT domain, robots_content, delay FROM crawldb.site")
    db_response = cur.fetchall()
    #print(db_response)
    cur.close()
    response = make_response(jsonify(db_response), 200)

    return response

In [7]:
@app.route('/new_url', methods=['GET'])
@basic_auth.login_required
def new_url():
    cur = conn.cursor()
    # return first link in frontier
    cur.execute("SELECT id, url FROM crawldb.page WHERE page_type_code = 'FRONTIER' LIMIT 1")

    db_response = cur.fetchall()
    #print(db_response)
    page_id = db_response[0][0]
    #print(page_id)
    cur.execute(f"UPDATE crawldb.page SET page_type_code = 'PARSING' WHERE id = {page_id};")
    cur.close()
    link = {"link": db_response[0][1]}
    response = make_response(jsonify(link), 200)
    return response

In [8]:
@app.route('/hashes', methods=['GET'])
@basic_auth.login_required
def hashes():
    cur = conn.cursor()
    # return all hashes
    cur.execute("SELECT hash FROM crawldb.page")
    db_response = cur.fetchall()
    #print(db_response)

    response = make_response(jsonify(db_response), 200)

    return response

In [9]:
@app.route('/save_domain', methods=['POST'])
@basic_auth.login_required
def save_domain(json):
    #input_json = request.get_json()
    input_json = json
    cur = conn.cursor()
    cur.execute(f"SELECT id FROM crawldb.site WHERE domain = '{input_json.get('domain')}'")
    site_id = cur.fetchone()

    if site_id is not None:
        site_id = site_id[0]

    #print(site_id)
    if site_id is None:
        if input_json.get('sitemap_host_content')[-1]:
            sitemap = input_json.get('sitemap_host_content')[0]
        else:
            sitemap = None
            
        if input_json.get('robot_delay') is not None:
            robot_delay = {input_json.get('robot_delay')}
        else:
            robot_delay = 5
        
        cur.execute('INSERT INTO crawldb.site '
                    '(domain, robots_content, sitemap_content, delay) '
                    f"VALUES ( '{input_json.get('domain')}', '{input_json.get('robot_txt_content')}', "
                    f"'{sitemap}', {robot_delay} ) ON CONFLICT (domain) DO NOTHING RETURNING id")
        site_id = cur.fetchone()
        if site_id is not None:
            site_id = site_id[0]

        # add links from sitemap into FRONTIER
        links = input_json.get('sitemap_content_links')
        filtered_links = []
        for link in links:
            parsed_link = urlparse(link)
            if parsed_link.netloc.endswith(".gov.si"):
                filtered_links.append(link)
        if len(filtered_links) > 0:
            url_query = "INSERT INTO crawldb.page (url, page_type_code) VALUES " + ",".join(
                [f"('{link}', 'FRONTIER')" for link in filtered_links]) + " ON CONFLICT (url) DO NOTHING;"
        # print(query)
            cur.execute(url_query)


    #print(site_id)

    cur.close()

    response = make_response(jsonify({"site_id": site_id}), 200)

    return response

In [10]:
@app.route('/save_page', methods=['POST'])
@basic_auth.login_required
def save_page():
    all_json = request.get_json()

    cur = conn.cursor()

    if 'error' in all_json:
        cur.execute('DELETE FROM crawldb.page WHERE'
                    f" url = '{all_json.get('url')}'")
    else:
        try:
            input_json = all_json[0][1]
            site_json = all_json[0][0]
            
            try:
                response = save_domain(site_json)
                site_id = response.get_json().get('site_id')
            except Exception as e:
                print("Error when saving domain: ", e)
                site_id = None
            
        except Exception as e:
            print("Error when parsing JSON :", e)
            print(all_json)
            input_json = all_json
        #print(site_json)
        
        #print(site_id)

        if input_json.get('pageType') == "HTML":

            cur.execute("SELECT id, hash FROM crawldb.page")
            hashes = cur.fetchall()
            #print(hashes)

            new_hash = input_json.get('hash')

            id_of_original = None

            duplicate = False
            for hash in hashes:
                if hash[1] == new_hash:
                    duplicate = True
                    id_of_original = hash[0]

            #print(duplicate, id_of_original)
            if duplicate:
                cur.execute('UPDATE crawldb.page SET'
                            f" page_type_code = 'DUPLICATE', "
                            f" site_id = {site_id}, "
                            f" http_status_code = {input_json.get('httpStatusCode')}, "
                            f" accessed_time = '{input_json.get('accessedTime')}', " 
                            f" hash = '{input_json.get('hash')}' WHERE "
                            f" url = '{input_json.get('url')}' RETURNING id" )

                dup_page_id = cur.fetchone()[0]

                cur.execute(f"INSERT INTO link (from_page, to_page) VALUES ({id_of_original},{dup_page_id})")
            else:
                """
                cur.execute('UPDATE crawldb.page SET'
                            f" page_type_code = '{input_json.get('pageType')}', "
                            f" site_id = {site_id}, "
                            f" html_content = '{input_json.get('html')}', "
                            f" http_status_code = {input_json.get('httpStatusCode')}, "
                            f" accessed_time = '{input_json.get('accessedTime')}', " 
                            f" hash = '{input_json.get('hash')}' WHERE "
                            f" url = '{input_json.get('url')}' RETURNING id")
                """
                #print(input_json)
                cur.execute("UPDATE crawldb.page SET page_type_code = %s, site_id = %s, html_content = %s, http_status_code = %s, accessed_time = %s, hash = %s WHERE url = %s RETURNING id",  \
                (input_json.get('pageType'), str(site_id), input_json.get('html'), str(input_json.get('httpStatusCode')), input_json.get('accessedTime'), input_json.get('hash'), input_json.get('url')))
                
                db_result = cur.fetchone()
                page_id = db_result[0]

                img_query = "INSERT INTO image (page_id, content_type, filename, accessed_time) VALUES " + ",".join(
                    [f"({page_id}, '{img.get('contentType')}', '{img.get('filename')}', '{img.get('accessedTime')}' )" for img in input_json.get('img')]) + ";"

                cur.execute(img_query)

                links = input_json.get('links')
                filtered_links = []
                for link in links:
                    parsed_link = urlparse(link)
                    if parsed_link.netloc.endswith(".gov.si"):
                        filtered_links.append(link)
                if len(filtered_links) > 0:
                    url_query = "INSERT INTO crawldb.page (url, page_type_code) VALUES " + ",".join([f"('{link}', 'FRONTIER')" for link in filtered_links]) + " ON CONFLICT (url) DO NOTHING;"
                #print(query)
                cur.execute(url_query)

        elif input_json.get('pageType') == "BINARY":
            if input_json.get('pageTypeCode') in ["DOC", "DOCX", "PDF", "PPT", "PPTX", "XLS", "XLSX", "WEBP", "BMP", "GIF", "SVG", "PNG", "JPG", "JPEG"]:
                cur.execute('UPDATE crawldb.page SET'
                            f" page_type_code = '{input_json.get('pageType')}', "
                            f" site_id = {site_id}, "
                            f" http_status_code = {input_json.get('httpStatusCode')}, "
                            f" accessed_time = '{input_json.get('accessedTime')}', "
                            f" hash = '{input_json.get('hash')}' WHERE "
                            f" url = '{input_json.get('url')}' RETURNING id")
                db_result = cur.fetchone()
                page_id = db_result[0]
                #print(page_id)

                cur.execute("INSERT INTO crawldb.page_data (page_id, data_type_code) "
                            f"VALUES ({page_id}, '{input_json.get('pageTypeCode')}' )")
            else:
                cur.execute('DELETE FROM crawldb.page WHERE'
                            f" url = '{input_json.get('url')}'")
        #except:
        #    print("error")
        #    cur.execute('DELETE FROM crawldb.page WHERE'
        #                f" url = '{all_json[0][1].get('url')}'")



    cur.close()

    return make_response("SUCCESS", 200)

In [ ]:
if __name__ == "__main__":
    app.run(ssl_context='adhoc', host="0.0.0.0")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:5000
 * Running on https://172.17.0.4:5000
Press CTRL+C to quit
172.17.0.1 - - [08/Apr/2023 13:51:33] "GET /visited_domains HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:51:33] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:51:43] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:51:43] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:51:52] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:51:52] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:52:02] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:52:02] "GET /visited_domains HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:52:02] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:52:03] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:52:10] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 13:52:11] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr

Error when parsing JSON : 1
[{'error': 'Message: unknown error: cannot find Chrome binary\nStacktrace:\nBacktrace:\n\t(No symbol) [0x006C37D3]\n\t(No symbol) [0x00658B81]\n\t(No symbol) [0x0055B36D]\n\t(No symbol) [0x00579190]\n\t(No symbol) [0x00576FC9]\n\t(No symbol) [0x005B1ED5]\n\t(No symbol) [0x005B1B2C]\n\t(No symbol) [0x005AB216]\n\t(No symbol) [0x00580D97]\n\t(No symbol) [0x0058253D]\n\tGetHandleVerifier [0x0093ABF2+2510930]\n\tGetHandleVerifier [0x00968EC1+2700065]\n\tGetHandleVerifier [0x0096C86C+2714828]\n\tGetHandleVerifier [0x00773480+645344]\n\t(No symbol) [0x00660FD2]\n\t(No symbol) [0x00666C68]\n\t(No symbol) [0x00666D4B]\n\t(No symbol) [0x00670D6B]\n\tBaseThreadInitThunk [0x779700F9+25]\n\tRtlGetAppContainerNamedObjectPath [0x77CD7BBE+286]\n\tRtlGetAppContainerNamedObjectPath [0x77CD7B8E+238]\n', 'method': 'process_url', 'url': 'https://www.gov.si/novice/2022-10-07-odlocitve-z-19-seje-vlade-s-podrocja-kmetijstva-gozdarstva-in-prehrane/'}]


172.17.0.1 - - [08/Apr/2023 14:06:26] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:28] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:29] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:31] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:31] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:36] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:36] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:39] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:40] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:43] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:43] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:47] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:47] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 14:06:51] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023

Error when parsing JSON : 1
[{'error': 'Message: unknown error: cannot find Chrome binary\nStacktrace:\nBacktrace:\n\t(No symbol) [0x006C37D3]\n\t(No symbol) [0x00658B81]\n\t(No symbol) [0x0055B36D]\n\t(No symbol) [0x00579190]\n\t(No symbol) [0x00576FC9]\n\t(No symbol) [0x005B1ED5]\n\t(No symbol) [0x005B1B2C]\n\t(No symbol) [0x005AB216]\n\t(No symbol) [0x00580D97]\n\t(No symbol) [0x0058253D]\n\tGetHandleVerifier [0x0093ABF2+2510930]\n\tGetHandleVerifier [0x00968EC1+2700065]\n\tGetHandleVerifier [0x0096C86C+2714828]\n\tGetHandleVerifier [0x00773480+645344]\n\t(No symbol) [0x00660FD2]\n\t(No symbol) [0x00666C68]\n\t(No symbol) [0x00666D4B]\n\t(No symbol) [0x00670D6B]\n\tBaseThreadInitThunk [0x779700F9+25]\n\tRtlGetAppContainerNamedObjectPath [0x77CD7BBE+286]\n\tRtlGetAppContainerNamedObjectPath [0x77CD7B8E+238]\n', 'method': 'process_url', 'url': 'https://www.gov.si/novice/2023-03-08-7-marec-dan-pomorstva-utrinki/'}]


...

172.17.0.1 - - [08/Apr/2023 19:50:31] "GET /new_url HTTP/1.1" 200 -
[2023-04-08 19:50:32,231] ERROR in app: Exception on /save_page [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/opt/conda/lib/python3.10/site-packages/flask_httpauth.py", line 172, in decorated
    return self.ensure_sync(f)(*args, **kwargs)
  File "/tmp/ipykernel_1031/189576649.py", line 72, in save_page
    cur.execute("UPDATE crawldb.page SET page_type_

Error when saving domain:  syntax error at or near "load"
LINE 51:     window.addEventListener('load', function() {
                                      ^



172.17.0.1 - - [08/Apr/2023 19:50:32] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:33] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:33] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:34] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:34] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:35] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:36] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:37] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:37] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:38] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:38] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:39] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:39] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:50:39] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023

Error when parsing JSON : 1
[{'error': "HTTPSConnectionPool(host='www.gov.si', port=443): Max retries exceeded with url: /novice/2021-12-03-predsednik-vlade-na-vrhu-srednjeevropske-pobude-sep/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022AD392D390>, 'Connection to www.gov.si timed out. (connect timeout=None)'))", 'method': 'process_url', 'url': 'https://www.gov.si/novice/2021-12-03-predsednik-vlade-na-vrhu-srednjeevropske-pobude-sep/'}]


172.17.0.1 - - [08/Apr/2023 19:59:46] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:46] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:47] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:47] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:48] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:48] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:48] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:50] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:50] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:51] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:51] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:51] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:51] "GET /new_url HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023 19:59:52] "POST /save_page HTTP/1.1" 200 -
172.17.0.1 - - [08/Apr/2023